In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Shanghai-Car-License-Plate-Auction-Price'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data = data['avg price']

In [4]:
data.to_csv('./Cleaned-Data.csv')

In [5]:
data = torch.from_numpy(np.array(data.tolist())).view(1,-1).to(device).float()

In [6]:
data_input = data[:1,:-1]
data_target = data[:1,1:]

In [7]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 512
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.lstm3 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t3,c_t3 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.linear(h_t3)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.linear(h_t3)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [8]:
model = Model().to(device)

In [9]:
criterion = MSELoss()

In [10]:
optimizer = LBFGS(model.parameters(),lr=0.8)

In [11]:
epochs = 100

In [12]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 10
        n = data_input.shape[1]
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].view(-1).cpu().detach().numpy()
        plt.figure(figsize=(12,7))
        plt.plot(np.arange(n),data_target.view(-1).cpu().detach().numpy(),'r')
        plt.plot(np.arange(n,n+future),preds[n:],'b','-')
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()